### Code-Zusammenfassung

Der folgende Code führt bestimmte Operationen im Zusammenhang mit der Datenbereinigung, resp. -Vorbereitung für das spätere maschinelle Lernen und Datenanalysen durch.

### Imports

<span style="font-family: Arial; font-size: 11pt;">

Der Code importiert die folgenden Bibliotheken:

- psycopg2: Eine Bibliothek zur Verbindung mit einer PostgreSQL-Datenbank.
- pandas: Eine Bibliothek zur Datenmanipulation und -analyse.
- sklearn.model_selection.train_test_split: Eine Funktion zum Aufteilen von Daten in Trainings- und Testsets.
- sklearn.tree.DecisionTreeRegressor: Ein Modellalgorithmus für Regressionsanalysen basierend auf Entscheidungsbäumen.
- sklearn.multioutput.MultiOutputRegressor: Eine Klasse zur Modellierung von mehreren Ausgaben in der Regressionsanalyse.
- sklearn.metrics.mean_squared_error: Eine Funktion zur Berechnung des mittleren quadratischen Fehlers.

#### Weitere Code-Operationen

Der Code führt auch weitere Operationen durch, die nicht explizit importierte Bibliotheken erfordern. Diese Operationen können Daten laden, Modelltraining und -evaluation durchführen, Berechnungen durchführen usw. Der genaue Inhalt dieser Operationen kann im Code selbst gefunden werden.

</span>

In [ ]:
import psycopg2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

#### Verbindung zur Datenbank herstellen

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    database="PSQL_ADSFS2023Gruppe15",
    user="ADSFS2023Gruppe15",
    password="ADS_FS_2023_G15!?"
)

#### SQL Queries für die DB Abfrage
#### Anschliessend die Daten aus den DB Tabellen lesen und in Dataframes speichern
#### Zusammenfassen aller Daten in ein einziges DataFrame

In [ ]:
query_tabelle = "SELECT team_id, mannschaft, punkte, tore FROM bundesliga_mannschaften"
query_resultate = "SELECT matchday, id_teamh, id_teamg, tore_teamh, tore_teamg FROM bundesliga_resultate"
query_shots_stats = "SELECT matchday, team_id, shots_total FROM bundesliga_shots_stats"
query_duel_stats = "SELECT matchday, team_id, duels_total, duels_won FROM bundesliga_duels_stats"
query_pass_stats = "SELECT matchday, team_id, pass_complete, pass_failed, pass_total, pass_percentage FROM bundesliga_pass_stats"

df_tabelle = pd.read_sql(query_tabelle, conn)
df_resultate = pd.read_sql(query_resultate, conn)
df_shots = pd.read_sql(query_shots_stats, conn)
df_duels = pd.read_sql(query_duel_stats, conn)
df_pass = pd.read_sql(query_pass_stats, conn)

merged_df = pd.merge(df_resultate, df_tabelle, left_on='id_teamh', right_on='team_id')
merged_df = pd.merge(merged_df, df_tabelle, left_on='id_teamg', right_on='team_id', suffixes=('_home', '_guest'))
merged_df = pd.merge(merged_df, df_shots, left_on=['id_teamh', 'matchday'], right_on=['team_id', 'matchday'])
merged_df = pd.merge(merged_df, df_shots, left_on=['id_teamg', 'matchday'], right_on=['team_id', 'matchday'])
merged_df = pd.merge(merged_df, df_duels, left_on=['id_teamh', 'matchday'], right_on=['team_id', 'matchday'])
merged_df = pd.merge(merged_df, df_duels, left_on=['id_teamg', 'matchday'], right_on=['team_id', 'matchday'])
merged_df = pd.merge(merged_df, df_pass, left_on=['id_teamh', 'matchday'], right_on=['team_id', 'matchday'])
merged_df = pd.merge(merged_df, df_pass, left_on=['id_teamg', 'matchday'], right_on=['team_id', 'matchday'])


merged_df_dummies = pd.get_dummies(merged_df, columns=['mannschaft_home', 'mannschaft_guest'])

#### Erstellen und Evaluieren eines Mehrfachausgabe-Entscheidungsbaum-Regressionsmodells

<span style="font-family: Arial; font-size: 11pt;">

1. Zuerst werden Eingabematrix `X` und Zielvektor `y` aus dem DataFrame `merged_df_dummies` erstellt. 
2. Dann werden diese Daten in Trainings- und Testdatensätze aufgeteilt, wobei 70% der Daten zum Training und 30% zum Testen verwendet werden.
3. Ein `MultiOutputRegressor` Modell, basierend auf einem `DecisionTreeRegressor`, wird erstellt.
4. Das Modell wird dann mit den Trainingsdaten trainiert und auf den Testdaten getestet.
5. Schliesslich wird der Mittlere Quadratische Fehler (MSE) berechnet, um die Leistung des Modells zu bewerten.


</span>

In [ ]:
X = merged_df_dummies.drop(['id_teamh', 'id_teamg', 'tore_teamh', 'tore_teamg', 'matchday'], axis=1)
y = merged_df_dummies[['tore_teamh', 'tore_teamg']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


multioutput = MultiOutputRegressor(DecisionTreeRegressor(random_state=42))
multioutput.fit(X_train, y_train)
y_pred = multioutput.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
print("MSE: {:.2f}".format(mse))